In [1]:
import time
import pandas as pd
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import os

In [2]:
adnimerge = pd.read_csv('data-CNvsMCIvsAD.csv') # Read all data
adnimerge.head()

,RID,Label,Visit_Code,RBM Sample ID,Sample_Received_Date,Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Alpha-1-Antichymotrypsin (AACT) (ug/ml),Alpha-1-Antitrypsin (AAT) (mg/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,3,AD,bl,KA8000CL-02,13-Sep-05,1.17609,0.07918,2.33445,0.44716,2.00432,...,1.3,1.34242,3.41330,0.55630,2.68305,2.98722,2.97035,2.96379,1.23045,1.14613
1,8,CN,bl,GA8000D4-02,20-Sep-05,0.97313,0.27875,3.12710,0.50515,1.97772,...,2.1,0.75587,3.46982,0.32222,2.67943,2.86153,2.81954,2.88986,1.41497,1.89763
2,8,CN,m12,HA803FG6-02,21-Sep-06,0.94448,0.07918,3.16137,0.49136,1.96848,...,3.4,0.69897,3.37475,0.44716,2.50651,2.88423,2.87099,2.97081,1.41497,1.83251
3,10,AD,bl,AA80013M-03,11-Nov-05,1.14613,0.07918,2.41664,0.38021,2.10721,...,1.3,1.23045,3.35984,0.41497,2.40654,2.88024,2.74741,2.88536,1.23045,1.51851
4,14,CN,bl,BA802V7G-02,7-Nov-05,1.00000,0.07918,2.52244,0.47712,1.93952,...,1.0,1.14613,3.41830,0.07918,2.68124,2.68124,2.73400,2.90526,1.30103,1.77815


In [3]:
adnimerge['Label'].value_counts() # Review target distribution again

MCI    136
CN     112
AD     112
Name: Label, dtype: int64

In [4]:
# View all data sizes
adnimerge.shape

(360, 151)

In [5]:
# Missing value count
for col in adnimerge.columns:
    print(col, adnimerge[col].isnull().sum())

RID 0
Label 0
Visit_Code 0
RBM Sample ID 0
Sample_Received_Date 0
Alpha-1-Microglobulin (A1Micro) (ug/ml) 0
Alpha-2-Macroglobulin (A2Macro) (mg/mL) 0
Alpha-1-Antichymotrypsin (AACT) (ug/ml) 0
Alpha-1-Antitrypsin (AAT) (mg/mL) 0
Angiotensin-Converting Enzyme (ACE) (ng/ml) 0
Adiponectin (ug/mL) 0
Alpha-Fetoprotein (AFP) (ng/mL) 0
Agouti-Related Protein (AGRP) (pg/mL) 0
Angiopoietin-2 (ANG-2) (ng/mL) 0
Angiotensinogen (ng/mL) 0
Apolipoprotein A-I (Apo A-I) (mg/mL) 0
Apolipoprotein A-II (Apo A-II) (ng/ml) 0
Apolipoprotein A-IV (Apo A-IV) (ug/ml) 0
Apolipoprotein B (Apo B) (ug/ml) 0
Apolipoprotein C-I (Apo C-I) (ng/ml) 0
Apolipoprotein C-III (Apo C-III) (ug/mL) 0
Apolipoprotein D (Apo D) (ug/ml) 0
Apolipoprotein E (Apo E) (ug/ml) 0
Apolipoprotein H (Apo H) (ug/mL) 0
AXL Receptor Tyrosine Kinase (AXL) (ng/mL) 0
Beta-2-Microglobulin (B2M) (ug/mL) 0
Brain-Derived Neurotrophic Factor (BDNF) (ng/mL) 0
B Lymphocyte Chemoattractant (BLC) (pg/ml) 0
Bone Morphogenetic Protein 6 (BMP-6) (ng/mL) 0
Bra

In [6]:
data=adnimerge

In [7]:
cols_with_na=[]

for column in data:
    if data[column].isnull().any()==True:
        cols_with_na.append(column)

# Average Fill
meandf = data.copy()
for col in cols_with_na:
    if data[col].dtype=='float64':
        meandf[col] = data[col].fillna(data[col].mean())
    else: #For non numeric data
        meandf[col] = data[col].fillna(data[col].mode()[0])
        
np.any(pd.isnull(meandf)) # If `False`，Confirm that Meandf currently has no Nan

False

In [8]:
meandf.head()

,RID,Label,Visit_Code,RBM Sample ID,Sample_Received_Date,Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Alpha-1-Antichymotrypsin (AACT) (ug/ml),Alpha-1-Antitrypsin (AAT) (mg/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,3,AD,bl,KA8000CL-02,13-Sep-05,1.17609,0.07918,2.33445,0.44716,2.00432,...,1.3,1.34242,3.41330,0.55630,2.68305,2.98722,2.97035,2.96379,1.23045,1.14613
1,8,CN,bl,GA8000D4-02,20-Sep-05,0.97313,0.27875,3.12710,0.50515,1.97772,...,2.1,0.75587,3.46982,0.32222,2.67943,2.86153,2.81954,2.88986,1.41497,1.89763
2,8,CN,m12,HA803FG6-02,21-Sep-06,0.94448,0.07918,3.16137,0.49136,1.96848,...,3.4,0.69897,3.37475,0.44716,2.50651,2.88423,2.87099,2.97081,1.41497,1.83251
3,10,AD,bl,AA80013M-03,11-Nov-05,1.14613,0.07918,2.41664,0.38021,2.10721,...,1.3,1.23045,3.35984,0.41497,2.40654,2.88024,2.74741,2.88536,1.23045,1.51851
4,14,CN,bl,BA802V7G-02,7-Nov-05,1.00000,0.07918,2.52244,0.47712,1.93952,...,1.0,1.14613,3.41830,0.07918,2.68124,2.68124,2.73400,2.90526,1.30103,1.77815


In [9]:
# Map and encode the target
mapping = {
           'CN': 0,
           'MCI': 1,
           'AD': 2
}
meandf['Label'] =meandf['Label'].map(mapping)
meandf.head()

,RID,Label,Visit_Code,RBM Sample ID,Sample_Received_Date,Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Alpha-1-Antichymotrypsin (AACT) (ug/ml),Alpha-1-Antitrypsin (AAT) (mg/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,3,2,bl,KA8000CL-02,13-Sep-05,1.17609,0.07918,2.33445,0.44716,2.00432,...,1.3,1.34242,3.41330,0.55630,2.68305,2.98722,2.97035,2.96379,1.23045,1.14613
1,8,0,bl,GA8000D4-02,20-Sep-05,0.97313,0.27875,3.12710,0.50515,1.97772,...,2.1,0.75587,3.46982,0.32222,2.67943,2.86153,2.81954,2.88986,1.41497,1.89763
2,8,0,m12,HA803FG6-02,21-Sep-06,0.94448,0.07918,3.16137,0.49136,1.96848,...,3.4,0.69897,3.37475,0.44716,2.50651,2.88423,2.87099,2.97081,1.41497,1.83251
3,10,2,bl,AA80013M-03,11-Nov-05,1.14613,0.07918,2.41664,0.38021,2.10721,...,1.3,1.23045,3.35984,0.41497,2.40654,2.88024,2.74741,2.88536,1.23045,1.51851
4,14,0,bl,BA802V7G-02,7-Nov-05,1.00000,0.07918,2.52244,0.47712,1.93952,...,1.0,1.14613,3.41830,0.07918,2.68124,2.68124,2.73400,2.90526,1.30103,1.77815


In [10]:
meandf.isnull().sum()

RID                                                 0
Label                                               0
Visit_Code                                          0
RBM Sample ID                                       0
Sample_Received_Date                                0
                                                   ..
Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL)    0
Vascular Endothelial Growth Factor (VEGF (pg/mL)    0
Vitronectin (ug/ml)                                 0
Vitamin K-Dependent Protein S (VKDPS) (ug/ml)       0
von Willebrand Factor (vWF) (ug/mL)                 0
Length: 151, dtype: int64

In [11]:
X=meandf.drop(['RID','Label','Visit_Code','RBM Sample ID','Sample_Received_Date'],axis=1)#Separate non data information such as RID, tags, and testing time
y=meandf['Label']#Label information
X

,Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Alpha-1-Antichymotrypsin (AACT) (ug/ml),Alpha-1-Antitrypsin (AAT) (mg/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),Adiponectin (ug/mL),Alpha-Fetoprotein (AFP) (ng/mL),Agouti-Related Protein (AGRP) (pg/mL),Angiopoietin-2 (ANG-2) (ng/mL),Angiotensinogen (ng/mL),...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,1.17609,0.07918,2.33445,0.44716,2.00432,0.93450,0.27875,2.09342,0.74819,0.47712,...,1.3,1.34242,3.41330,0.55630,2.68305,2.98722,2.97035,2.96379,1.23045,1.14613
1,0.97313,0.27875,3.12710,0.50515,1.97772,0.98677,0.50515,1.83251,0.66276,0.51851,...,2.1,0.75587,3.46982,0.32222,2.67943,2.86153,2.81954,2.88986,1.41497,1.89763
2,0.94448,0.07918,3.16137,0.49136,1.96848,1.04139,0.34242,2.18184,1.04139,0.59106,...,3.4,0.69897,3.37475,0.44716,2.50651,2.88423,2.87099,2.97081,1.41497,1.83251
3,1.14613,0.07918,2.41664,0.38021,2.10721,1.04139,0.07918,1.79588,0.65321,1.34242,...,1.3,1.23045,3.35984,0.41497,2.40654,2.88024,2.74741,2.88536,1.23045,1.51851
4,1.00000,0.07918,2.52244,0.47712,1.93952,0.81291,0.59106,2.02531,0.44716,1.20412,...,1.0,1.14613,3.41830,0.07918,2.68124,2.68124,2.73400,2.90526,1.30103,1.77815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,1.07918,0.07918,2.34635,0.38021,1.99123,0.88081,0.38021,2.23045,0.53148,1.23045,...,3.9,0.97772,3.35984,0.55630,2.61278,2.79657,2.82802,2.88986,1.32222,1.23045
356,1.14613,-0.05061,2.50920,0.43136,2.05690,1.04139,-0.20066,3.29885,0.57978,0.79934,...,3.6,1.47712,3.52892,0.23045,2.60531,2.75740,2.78247,2.88986,1.32222,1.53148
357,1.00000,-0.07572,2.39967,0.41497,1.91908,0.77815,0.50515,1.85733,0.51851,0.99564,...,1.5,1.20412,3.32838,0.30103,2.47567,2.73239,2.61490,2.94596,1.32222,1.32222
358,0.96379,0.00000,2.39270,0.43136,1.90309,0.63347,0.14613,2.18469,0.55630,0.27875,...,2.4,1.23045,3.55023,0.00000,2.59879,2.81624,2.83569,2.95952,1.23045,0.60206


In [12]:
print(type(X))
print(type(y))
print(y)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
0      2
1      0
2      0
3      2
4      0
      ..
355    2
356    2
357    2
358    1
359    2
Name: Label, Length: 360, dtype: int64


In [13]:
#Data Z-score standardization

zscore = preprocessing.StandardScaler()
standard_X = zscore.fit_transform(X)
standard_X = pd.DataFrame(standard_X,index=X.index,columns=X.columns)
standard_X

,Alpha-1-Microglobulin (A1Micro) (ug/ml),Alpha-2-Macroglobulin (A2Macro) (mg/mL),Alpha-1-Antichymotrypsin (AACT) (ug/ml),Alpha-1-Antitrypsin (AAT) (mg/mL),Angiotensin-Converting Enzyme (ACE) (ng/ml),Adiponectin (ug/mL),Alpha-Fetoprotein (AFP) (ng/mL),Agouti-Related Protein (AGRP) (pg/mL),Angiopoietin-2 (ANG-2) (ng/mL),Angiotensinogen (ng/mL),...,Thrombopoietin (ng/mL),TNF-Related Apoptosis-Inducing Ligand Re (ng/mL),Serotransferrin (Transferrin) (mg/dl),Thyroid-Stimulating Hormone (TSH) (uIU/mL),Transthyretin (TTR) (mg/dl),Vascular Cell Adhesion Molecule-1 (VCAM- (ng/mL),Vascular Endothelial Growth Factor (VEGF (pg/mL),Vitronectin (ug/ml),Vitamin K-Dependent Protein S (VKDPS) (ug/ml),von Willebrand Factor (vWF) (ug/mL)
0,0.905103,0.498143,-0.970129,0.201408,0.093271,0.717736,-0.393544,-0.426616,0.789731,-0.930485,...,-0.890134,1.032386,-0.037821,1.036850,1.261487,1.143723,1.366537,0.217757,-0.661018,-1.085362
1,-0.979887,2.507987,3.973452,0.818072,-0.074566,0.917211,0.401352,-1.440666,0.213794,-0.870168,...,-0.026799,-2.537929,0.695229,0.283618,1.230314,-0.044293,0.157665,-0.825995,1.479532,0.725325
2,-1.245974,0.498143,4.187186,0.671429,-0.132868,1.125655,-0.169997,-0.082964,2.766374,-0.764441,...,1.376119,-2.884278,-0.537805,0.685655,-0.258753,0.170267,0.570081,0.316866,1.479532,0.568422
3,0.626849,0.498143,-0.457528,-0.510536,0.742474,1.125655,-1.094238,-1.583032,0.149412,0.330515,...,-0.890134,0.350828,-0.731184,0.582073,-1.119626,0.132553,-0.420520,-0.889527,-0.661018,-0.188139
4,-0.730332,0.498143,0.202323,0.520002,-0.315596,0.253718,0.702983,-0.691332,-1.239699,0.128971,...,-1.213884,-0.162426,0.027028,-0.498446,1.245901,-1.748385,-0.528012,-0.608576,0.157755,0.437446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.005052,0.498143,-0.895911,-0.510536,0.010677,0.512841,-0.037316,0.105963,-0.671246,0.167341,...,1.915703,-1.187534,-0.731184,1.036850,0.656371,-0.658291,0.225639,-0.825995,0.403572,-0.882199
356,0.626849,-0.808956,0.119748,0.033391,0.425034,1.125655,-2.076763,4.258394,-0.345625,-0.460915,...,1.591953,1.852302,1.461742,-0.011683,0.592044,-1.028524,-0.139483,-0.825995,0.403572,-0.156888
357,-0.730332,-1.061835,-0.563366,-0.140899,-0.444566,0.121065,0.401352,-1.344201,-0.758685,-0.174847,...,-0.674300,0.190557,-1.139212,0.215432,-0.524326,-1.264918,-1.482702,-0.033969,0.403572,-0.661086
358,-1.066632,-0.299269,-0.606836,0.033391,-0.545457,-0.431070,-0.859176,-0.071888,-0.503919,-1.219570,...,0.296951,0.350828,1.738127,-0.753235,0.535898,-0.472372,0.287121,0.157473,-0.661018,-2.396261


In [14]:
X=standard_X 
X.shape

(360, 146)

In [15]:
#Convert to TensorFlow form（width, height, img_num_channels）
data_ndim=1
X = np.array(X).reshape(X.shape[0],X.shape[1],data_ndim)
print(X.shape)
inputs=X
targets=y

(360, 146, 1)


In [16]:
# Model configuration
batch_size = 32
img_width, img_height, img_num_channels = 1, 146, 1
loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 9
optimizer = Adam()
verbosity = 1
num_folds = 5

In [17]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

realacc_per_fold = []
recall_per_fold = []
precision_per_fold = []
f1_per_fold = []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

In [18]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the 1D-CNN model architecture
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64, kernel_size=(3,), padding='same',
                                activation=tf.keras.layers.LeakyReLU(alpha=0.001), input_shape=(inputs.shape[1], 1)),
        tf.keras.layers.Conv1D(filters=128, kernel_size=(3,), padding='same',
                                activation=tf.keras.layers.LeakyReLU(alpha=0.001)),
        tf.keras.layers.Conv1D(filters=256, kernel_size=(3,), padding='same',
                                activation=tf.keras.layers.LeakyReLU(alpha=0.001)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=512, activation=tf.keras.layers.LeakyReLU(alpha=0.001)),
        tf.keras.layers.Dense(units=120, activation=tf.keras.layers.LeakyReLU(alpha=0.001)),
        tf.keras.layers.Dense(units=84, activation=tf.keras.layers.LeakyReLU(alpha=0.001)),
        tf.keras.layers.Dense(units=3, activation='softmax'),
    ])
    model.compile(tf.keras.optimizers.Adam(lr=0.0005),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['sparse_categorical_accuracy'])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
                batch_size=batch_size,
                epochs=no_epochs,
                verbose=verbosity)
    
    # Generate generalization metrics
    y_pred = model.predict(inputs[test])
    test_predictions = np.argmax(y_pred, axis=1)
    y_true=targets[test].astype('uint8')
    
    acc = accuracy_score(y_true,test_predictions)
    recall = recall_score(y_true,test_predictions,average='macro')
    precision = precision_score(y_true,test_predictions,average='macro')
    f1 = f1_score(y_true,test_predictions,average='macro')

    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; Acc: {acc}; precision: {precision}; recall: {recall}; f1_score: {f1}')
    
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    realacc_per_fold.append(acc) 
    recall_per_fold.append(recall) 
    precision_per_fold.append(precision) 
    f1_per_fold.append(f1) 
    
    # Increase fold number
    fold_no = fold_no + 1


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/9
9/9 [==============================] - 1s 110ms/step - loss: 1.0849 - sparse_categorical_accuracy: 0.3585
Epoch 2/9
9/9 [==============================] - 1s 109ms/step - loss: 0.7853 - sparse_categorical_accuracy: 0.7350
Epoch 3/9
9/9 [==============================] - 1s 109ms/step - loss: 0.2828 - sparse_categorical_accuracy: 0.9235
Epoch 4/9
9/9 [==============================] - 1s 110ms/step - loss: 0.1584 - sparse_categorical_accuracy: 0.9587
Epoch 5/9
9/9 [==============================] - 1s 111ms/step - loss: 0.0619 - sparse_categorical_accuracy: 0.9917
Epoch 6/9
9/9 [==============================] - 1s 109ms/step - loss: 0.0186 - sparse_categorical_accuracy: 1.0000
Epoch 7/9
9/9 [==============================] - 1s 112ms/step - loss: 0.0113 - sparse_categorical_accuracy: 0.9985
Epoch 8/9
9/9 [==============================] - 1s 109ms/step - loss: 0.0185 - sparse_cate

In [21]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Acc: {np.mean(realacc_per_fold)}')
print(f'> precision: {np.mean(precision_per_fold)}')
print(f'> recall: {np.mean(recall_per_fold)}')
print(f'> f1_score: {np.mean(f1_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.6785964965820312 - Accuracy: 80.55555820465088%
------------------------------------------------------------------------
> Fold 2 - Loss: 1.0548101663589478 - Accuracy: 80.55555820465088%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.7189120054244995 - Accuracy: 88.88888955116272%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.6285126805305481 - Accuracy: 81.94444179534912%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.8168002367019653 - Accuracy: 81.94444179534912%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 82.77777791023254 (+- 3.11804760991237)
> Loss: 0.7795263171195984
> Acc: 0.8277777777777778
> pr